In [ ]:
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
mpl.rcParams["figure.dpi"] = 300
from skimage.io import imread

plt.style.use("dark_background")
plt.rcParams["figure.figsize"] = [12, 8]
plt.rcParams["figure.dpi"] = 100  # 200 e.g. is really fine, but slower
plt.style.use("classic")
plt.style.use("seaborn-white")

plt.rcParams["figure.figsize"] = [12, 8]
plt.rcParams["figure.dpi"] = 100  # 200 e.g. is really fine, but slower

mpl.rcParams["pdf.fonttype"] = 42
mpl.rcParams["ps.fonttype"] = 42
plt.rcParams["font.family"] = "Arial"

import re

import pandas as pd
import scanpy as sc

sc.set_figure_params(dpi=200)


import seaborn as sns
import skimage
from morphometrics.explore.cluster import cluster_features
from morphometrics.explore.dimensionality_reduction import pca
from morphometrics.utils.anndata_utils import table_to_anndata
from skimage.measure import label
from tqdm import tqdm

rng = np.random.default_rng(42)
import scipy.spatial.distance as distance


def colorFader(
    c1, c2, mix=0
):  # fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
    c1 = np.array(mpl.colors.to_rgb(c1))
    c2 = np.array(mpl.colors.to_rgb(c2))
    return mpl.colors.to_hex((1 - mix) * c1 + mix * c2)


cm = 1 / 2.54  # centimeters in inches

In [ ]:
# Color palette
color_pink_midnight_blue = matplotlib.colors.LinearSegmentedColormap.from_list(
    "", ["#b3107a", "#f7f7f7", "#191970"]
)

In [ ]:
# Load data + Pseudotime
measurement_data = sc.read_h5ad(
    "h5_repro/actin_morphometrics_all_panel_defg_12_07_2024.h5ad"
)

# pick starting cell for pseudotime
measurement_data.uns["iroot"] = np.flatnonzero(measurement_data.obs["leiden"] == "2")[9]
sc.tl.diffmap(measurement_data)
sc.tl.dpt(measurement_data)
mean_age = measurement_data.obs.groupby("leiden")["Day"].mean()

In [ ]:
# PAGA Path complete
sc.pl.paga_path(
    measurement_data,
    n_avg=50,
    nodes=["2", "6", "3", "7", "5", "0", "1", "4"],
    keys=measurement_data.var_names,
    color_maps_annotations={"dpt_pseudotime": "Greys", "Day": "Greys"},
    annotations=("dpt_pseudotime", "Day"),
    normalize_to_zero_one=True,
    color_map=color_pink_midnight_blue,
    save="_actin_complete_plot_zero_one.pdf",
)

In [ ]:
plot_measueremnts = [
    "surface_area",
    "area",
    "convex_area",
    "bbox_area",
    "curvature_mean",
    "curvature_stdev",
    "axis_minor_length",
    "axis_major_length",
]

In [ ]:
# PAGA Path sub
sc.pl.paga_path(
    measurement_data,
    n_avg=50,
    nodes=["2", "6", "3", "7", "5", "0", "1", "4"],
    keys=plot_measueremnts,
    color_maps_annotations={
        "Day": "Greys",
    },
    annotations=("Day",),
    normalize_to_zero_one=True,
    color_map=color_pink_midnight_blue,
    save="_actin_main_plot_pink_blue_zero_one.pdf",
)

In [ ]:
import seaborn as sns

# plot major axis length
sns.set_context("paper", rc={"font.size": 8, "axes.titlesize": 8, "axes.labelsize": 8})
rc = {
    "figure.figsize": (10, 5),
    "axes.facecolor": "white",
    "axes.grid": False,
    "grid.color": ".8",
    "font.size": 5,
}
plt.rcParams.update(rc)
cm = 1 / 2.54  # centimeters in inches

fig, ax = plt.subplots(figsize=(3, 2))
sns.despine(left=True, bottom=True, right=True)
measurement_data.obs["Axis length ratio"] = measurement_data.obs[
    "Axis_length_ratio_raw"
]
sns.boxplot(
    x="Day",
    y="Axis length ratio",
    data=measurement_data.obs,
    palette=palette,
    hue="condition",
    showfliers=False,
).legend(loc="center left", bbox_to_anchor=(1.0, 0.5), fontsize=16)
plt.savefig("figures/axis_length_ratio_boxplot.pdf", bbox_inches="tight")

In [ ]:
measurement_data.write_h5ad(
    f"h5_repro/actin_morphometrics_all_panel_defg_12_07_2024_pseudotime.h5ad"
)